In [61]:
import numpy as np 
import pandas as pd 
import fastf1
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

**GET DATA**

In [2]:
# Get data from entire 2024 season
# fastf1.get_event_schedule(2024,include_testing=False,backend='fastf1')

races24 = []
for race in range(1,25):
    session = fastf1.get_session(2024, race, 'R')
    session.load()
    country = session.event["Country"]
    race24 = session.results[["Abbreviation","TeamName","Position"]]
    race24["Country"] = country
    races24.append(race24)

races24 = pd.concat(races24, ignore_index=True)
# print(races24)

req         WARNING 	DEFAULT CACHE ENABLED! (3.25 GB) C:\Users\jenni\AppData\Local\Temp\fastf1
core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '55', '16', '63', '4', '44', '81', '14', '18', '24', '20', '3', '22', '2

In [ ]:
# Get data from entire 2023 season
# fastf1.get_event_schedule(2023,include_testing=False)

races23 = []
for race in range(1,23):
    session = fastf1.get_session(2023, race, 'R')
    session.load()
    country = session.event["Country"]
    race23 = session.results[["Abbreviation","TeamName","Position"]]
    race23["Country"] = country
    races23.append(race23)

races23 = pd.concat(races23, ignore_index=True)
# print(races23)

,RoundNumber,Country,Location,OfficialEventName,EventDate,EventName,EventFormat,Session1,Session1Date,Session1DateUtc,...,Session3,Session3Date,Session3DateUtc,Session4,Session4Date,Session4DateUtc,Session5,Session5Date,Session5DateUtc,F1ApiSupport
1,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2023,2023-03-05,Bahrain Grand Prix,conventional,Practice 1,2023-03-03 14:30:00+03:00,2023-03-03 11:30:00,...,Practice 3,2023-03-04 14:30:00+03:00,2023-03-04 11:30:00,Qualifying,2023-03-04 18:00:00+03:00,2023-03-04 15:00:00,Race,2023-03-05 18:00:00+03:00,2023-03-05 15:00:00,True
2,2,Saudi Arabia,Jeddah,FORMULA 1 STC SAUDI ARABIAN GRAND PRIX 2023,2023-03-19,Saudi Arabian Grand Prix,conventional,Practice 1,2023-03-17 16:30:00+03:00,2023-03-17 13:30:00,...,Practice 3,2023-03-18 16:30:00+03:00,2023-03-18 13:30:00,Qualifying,2023-03-18 20:00:00+03:00,2023-03-18 17:00:00,Race,2023-03-19 20:00:00+03:00,2023-03-19 17:00:00,True
3,3,Australia,Melbourne,FORMULA 1 ROLEX AUSTRALIAN GRAND PRIX 2023,2023-04-02,Australian Grand Prix,conventional,Practice 1,2023-03-31 12:30:00+10:00,2023-03-31 02:30:00,...,Practice 3,2023-04-01 12:30:00+10:00,2023-04-01 02:30:00,Qualifying,2023-04-01 16:00:00+10:00,2023-04-01 06:00:00,Race,2023-04-02 15:00:00+10:00,2023-04-02 05:00:00,True
4,4,Azerbaijan,Baku,FORMULA 1 AZERBAIJAN GRAND PRIX 2023,2023-04-30,Azerbaijan Grand Prix,sprint_shootout,Practice 1,2023-04-28 13:30:00+04:00,2023-04-28 09:30:00,...,Sprint Shootout,2023-04-29 12:30:00+04:00,2023-04-29 08:30:00,Sprint,2023-04-29 17:30:00+04:00,2023-04-29 13:30:00,Race,2023-04-30 15:00:00+04:00,2023-04-30 11:00:00,True
5,5,United States,Miami,FORMULA 1 CRYPTO.COM MIAMI GRAND PRIX 2023,2023-05-07,Miami Grand Prix,conventional,Practice 1,2023-05-05 14:00:00-04:00,2023-05-05 18:00:00,...,Practice 3,2023-05-06 12:30:00-04:00,2023-05-06 16:30:00,Qualifying,2023-05-06 16:00:00-04:00,2023-05-06 20:00:00,Race,2023-05-07 15:30:00-04:00,2023-05-07 19:30:00,True
6,6,Monaco,Monaco,FORMULA 1 GRAND PRIX DE MONACO 2023,2023-05-28,Monaco Grand Prix,conventional,Practice 1,2023-05-26 13:30:00+02:00,2023-05-26 11:30:00,...,Practice 3,2023-05-27 12:30:00+02:00,2023-05-27 10:30:00,Qualifying,2023-05-27 16:00:00+02:00,2023-05-27 14:00:00,Race,2023-05-28 15:00:00+02:00,2023-05-28 13:00:00,True
7,7,Spain,Barcelona,FORMULA 1 AWS GRAN PREMIO DE ESPAÑA 2023,2023-06-04,Spanish Grand Prix,conventional,Practice 1,2023-06-02 13:30:00+02:00,2023-06-02 11:30:00,...,Practice 3,2023-06-03 12:30:00+02:00,2023-06-03 10:30:00,Qualifying,2023-06-03 16:00:00+02:00,2023-06-03 14:00:00,Race,2023-06-04 15:00:00+02:00,2023-06-04 13:00:00,True
8,8,Canada,Montréal,FORMULA 1 PIRELLI GRAND PRIX DU CANADA 2023,2023-06-18,Canadian Grand Prix,conventional,Practice 1,2023-06-16 13:30:00-04:00,2023-06-16 17:30:00,...,Practice 3,2023-06-17 12:30:00-04:00,2023-06-17 16:30:00,Qualifying,2023-06-17 16:00:00-04:00,2023-06-17 20:00:00,Race,2023-06-18 14:00:00-04:00,2023-06-18 18:00:00,True
9,9,Austria,Spielberg,FORMULA 1 ROLEX GROSSER PREIS VON ÖSTERREICH 2023,2023-07-02,Austrian Grand Prix,sprint_shootout,Practice 1,2023-06-30 13:30:00+02:00,2023-06-30 11:30:00,...,Sprint Shootout,2023-07-01 12:00:00+02:00,2023-07-01 10:00:00,Sprint,2023-07-01 16:30:00+02:00,2023-07-01 14:30:00,Race,2023-07-02 15:00:00+02:00,2023-07-02 13:00:00,True
10,10,Great Britain,Silverstone,FORMULA 1 ARAMCO BRITISH GRAND PRIX 2023,2023-07-09,British Grand Prix,conventional,Practice 1,2023-07-07 12:30:00+01:00,2023-07-07 11:30:00,...,Practice 3,2023-07-08 11:30:00+01:00,2023-07-08 10:30:00,Qualifying,2023-07-08 15:00:00+01:00,2023-07-08 14:00:00,Race,2023-07-09 15:00:00+01:00,2023-07-09 14:00:00,True


**PRE-PROCESSING**

In [ ]:
# Make feature table
data = races24.iloc[:,[3,0,1,2]]
# print(data)
# Split features and target
X = data[["Country","TeamName","Abbreviation"]]
Y = data[["Position"]]

# One-hot encode
X_encoded = pd.get_dummies(X,columns=["Country","TeamName","Abbreviation"],drop_first=True)
# print(X_encoded)

# Binary encode - WIP
# import category_encoders as ce
# encoder = ce.BinaryEncoder(cols=["Country","TeamName","Abbreviation"])
# data_bin = encoder.fit_transform(data)
# print(data_bin)

# Train & test split
# Don't forget about validation

# Label encoding for XGB model
label_encoder = preprocessing.LabelEncoder()
X_le = pd.DataFrame()
X_le["Country"] = label_encoder.fit_transform(X["Country"])
X_le["TeamName"] = label_encoder.fit_transform(X["TeamName"])
X_le["Abbreviation"] = label_encoder.fit_transform(X["Abbreviation"])

print(X_le)


     Country  TeamName  Abbreviation
0          3         8            22
1          3         8            14
2          3         2            18
3          3         2            10
4          3         6            17
..       ...       ...           ...
474       18         3            11
475       18         7             9
476       18         4             3
477       18         9             4
478       18         8            14

[479 rows x 3 columns]


**Train the model**

In [78]:
model = RandomForestRegressor(n_estimators=100,random_state=42)
model.fit(X_encoded,Y)

from xgboost import XGBRegressor

xgb_model = XGBRegressor(objective='reg:squarederror', n_estimators=100)
xgb_model.fit(X_le, Y)

c:\Users\jenni\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=100,
             n_jobs=None, num_parallel_tree=None, ...)

**Test the model**

In [80]:
miami25 = pd.DataFrame({
    "Country": ["United States"] * 20,
    "Abbreviation": [
        "VER",  # Max Verstappen
        "TSU",  # Yuki Tsunoda
        "HAM",  # Lewis Hamilton
        "LEC",  # Charles Leclerc
        "RUS",  # George Russell
        "ANT",  # Andrea Kimi Antonelli
        "NOR",  # Lando Norris
        "PIA",  # Oscar Piastri
        "ALO",  # Fernando Alonso
        "STR",  # Lance Stroll
        "GAS",  # Pierre Gasly
        "DOO",  # Jack Doohan
        "ALB",  # Alex Albon
        "SAI",  # Carlos Sainz
        "LAW",  # Liam Lawson
        "HAD",  # Isack Hadjar
        "HUL",  # Nico Hülkenberg
        "BOR",  # Gabriel Bortoleto
        "BEA",  # Oliver Bearman
        "OCO"   # Esteban Ocon
    ],
    "TeamName": [
        "Red Bull Racing",
        "Red Bull Racing",
        "Ferrari",
        "Ferrari",
        "Mercedes",
        "Mercedes",
        "McLaren",
        "McLaren",
        "Aston Martin",
        "Aston Martin",
        "Alpine",
        "Alpine",
        "Williams",
        "Williams",
        "RB",
        "RB",
        "Kick Sauber",
        "Kick Sauber",
        "Haas F1 Team",
        "Haas F1 Team"
    ]
})

# Encode 2025 data
miami25_encoded = pd.get_dummies(miami25,columns=["Country","TeamName","Abbreviation"],drop_first=True)
# Get list of columns from training data
cols = X_encoded.columns.tolist()
# Fill 2025 data with training columns (like old drivers or tracks), setting them to False
miami25_encoded = miami25_encoded.reindex(columns=cols).fillna(False)
# print(miami25_encoded)

# Label encoding for XGB model
Y_le = pd.DataFrame()
Y_le["Country"] = label_encoder.fit_transform(miami25["Country"])
Y_le["TeamName"] = label_encoder.fit_transform(miami25["TeamName"])
Y_le["Abbreviation"] = label_encoder.fit_transform(miami25["Abbreviation"])

# print(Y_le)



C:\Users\jenni\AppData\Local\Temp\ipykernel_38084\1096783217.py:54: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  miami25_encoded = miami25_encoded.reindex(columns=cols).fillna(False)


Three options to test:
- 'normal' RandomForestRegressor
- XGB model (extreme gradiant boost) with one-hot encoded data (KEEP IN MIND TO TRAIN THE MODEL WITH CORRECTLY ENCODED DATA!)
- XGB model with label encoded data (KEEP IN MIND TO TRAIN THE MODEL WITH CORRECTLY ENCODED DATA!)

In [83]:
# predicted_positions = model.predict(miami25_encoded)
# predicted_positions = xgb_model.predict(miami25_encoded)
predicted_positions = xgb_model.predict(Y_le)


miami25["Predicted Position"] = predicted_positions

miami25_sorted = miami25.sort_values("Predicted Position").reset_index(drop=True)
miami25_sorted["FinalRank"] = miami25_sorted.index + 1
print(miami25_sorted)

          Country Abbreviation         TeamName  Predicted Position  FinalRank
0   United States          LEC          Ferrari            2.298326          1
1   United States          PIA          McLaren            2.818826          2
2   United States          HAM          Ferrari            3.366010          3
3   United States          NOR          McLaren            3.422979          4
4   United States          VER  Red Bull Racing            6.751069          5
5   United States          OCO     Haas F1 Team            7.370517          6
6   United States          TSU  Red Bull Racing            7.435099          7
7   United States          ALO     Aston Martin            8.046928          8
8   United States          STR     Aston Martin            8.121681          9
9   United States          BEA     Haas F1 Team            8.470482         10
10  United States          LAW               RB           10.089395         11
11  United States          HAD               RB     